In [4]:
import networkx as nx
import numpy as np
from functools import wraps
import time
# newman 快速算法


# 合并社团函数
def cluAssemble(self, other, currentCluList):
    currentCluList.remove(self)
    currentCluList.remove(other)
    for node in self:
        if node not in other:
            other.append(node)
    currentCluList.append(other)
    cluAfterAssemble = currentCluList
    return cluAfterAssemble


# 判断两个社团之间是否有边相连
def cluHasEdge(clu1, clu2, graph):
    for p1 in clu1:
        for p2 in clu2:
            if graph.has_edge(p1, p2):
                return True
    return False


# 从 path 中读取图
def load_graph(path):
    G = nx.Graph()
    with open(path) as text:
        for line in text:
            vertices = line.strip().split()[:2]
            sourcePoint = int(vertices[0])
            targetPoint = int(vertices[1])
            G.add_edge(sourcePoint, targetPoint)
    return G


# 计时器函数，该函数是一个装饰器 decorator
def fn_timer(function):
    @wraps(function)
    def function_timer(*args, **kwargs):
        t0 = time.time()
        result = function(*args, **kwargs)
        t1 = time.time()
        print("time:%s s" % (str(t1 - t0)))
        return result
    return function_timer
    
    
# 写入所属社团标号
def writeClu(G, currentCluList):
    cluID = 0
    for item in currentCluList:
        cluID += 1
        for item2 in item:
            G.nodes[item2]['groupID'] = str(cluID)
    # print(G.nodes(data=True))
    
    
# Q = 社团内部边数 / 网络总边数 - （社团内所有点度数之和 / 2 倍总边数）平方
def cal_Q(partition, G):
    m = len(G.edges())
    a = 0.0
    e = 0.0

    # 计算 eii
    for community in partition:
        eii = 0.0
        for i in community:
            for j in community:
                if G.has_edge(i, j):
                    eii += 1
        e += eii / (2 * m)
    # 计算 aij 的平方
    for community in partition:
        aij = 0.0
        for node in community:
            aij += len(list((G.neighbors(node))))
        a += (aij / (2 * m)) ** 2

    q = e - a
    return q


# 主类
class newmanFast:
    def __init__(self, graph):
        self.G = graph
        self.nodeList = self.G.nodes()
        self.cluList = []
        for i in self.nodeList:
            self.cluList.append([i])
        self.finalClu = []

    @fn_timer
    def execute(self):
        iterTime = 1
        # 先将每个节点看作一个社区，然后选择模块度增值最大的进行合并，直到所有社团变成一个社团为止
        print(self.cluList)
        maxQ = -float('Inf')
        # 只要社团列表长度不为 1
        while len(self.cluList) != 1:
            Q = -float('Inf')
            for cluFrom in self.cluList:
                thisClu = self.cluList.copy()
                thisClu.remove(cluFrom)
                for cluTo in thisClu:
                    if cluHasEdge(cluFrom, cluTo, self.G):
                        partition = cluAssemble(cluFrom.copy(), cluTo.copy(), self.cluList.copy())
                        thisQ = cal_Q(partition, self.G)
                        # print ("社团" + str (cluTo) + "q 值" + str (thisQ))
                        # 记录该轮最大 Q 和目标社团
                        if thisQ >= Q:
                            Q = thisQ
                            finalCluFrom = cluFrom
                            finalCluTo = cluTo
            cluAssemble(finalCluFrom.copy(), finalCluTo.copy(), self.cluList)
            print("该轮结束的划分结果:" + str(self.cluList))
            print("该轮结束时的模块度 Q:" + str(Q))
            print("################ 第" + str(iterTime) + "轮结束 ##################")
            file_object = open('%sres.txt' % str(graphName), 'a')
            file_object.write(str(iterTime) + '|Q:' + str(Q) + '|Result:' + str(self.cluList) + '\n')
            file_object.close()

            iterTime += 1
            if Q > maxQ:
                maxQ = Q
                iterRound = iterTime - 1
                self.finalClu = self.cluList.copy()
                writeClu(self.G, self.cluList)

        print("最大 Q 值出现在：第" + str(iterRound) + "轮。最大 Q 值为：" + str(maxQ))
        for clu in self.finalClu:
            print(sorted(clu))


if __name__ == "__main__":
    graphName = r"network"
    G = load_graph('%s.txt' % str(graphName))
    this = newmanFast(G)
    this.execute()
    nx.write_gml(G, '%sres.gml' % str(graphName))

[[1], [55], [102], [117], [143], [2], [42], [60], [75], [78], [3], [121], [132], [155], [170], [4], [175], [201], [228], [229], [5], [141], [205], [6], [242], [248], [249], [250], [7], [18], [21], [80], [93], [8], [233], [239], [240], [241], [9], [243], [10], [208], [213], [11], [140], [144], [156], [162], [12], [166], [193], [206], [217], [13], [182], [191], [194], [210], [14], [85], [116], [15], [234], [16], [221], [17], [211], [215], [220], [225], [20], [76], [133], [19], [238], [31], [22], [108], [110], [134], [196], [23], [53], [150], [24], [149], [178], [187], [188], [25], [165], [183], [184], [26], [146], [202], [27], [47], [164], [28], [29], [131], [30], [54], [32], [86], [159], [195], [203], [33], [83], [34], [41], [68], [69], [35], [36], [181], [37], [64], [99], [38], [71], [39], [40], [46], [43], [44], [100], [119], [124], [147], [45], [245], [48], [49], [97], [50], [111], [51], [52], [246], [56], [130], [160], [57], [126], [152], [58], [65], [59], [61], [192], [62], [237], 

ValueError: list.remove(x): x not in list